In [7]:
# Cell 1: 初始化运行参数
# ...existing code...
from pathlib import Path
from datetime import datetime

# 保留 Path 对象，便于后续路径拼接
PROJ_DIR = Path.home() / "mattergen"
RUN_TAG = f"lps_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
RESULTS_DIR = PROJ_DIR / "data" / "generated" / RUN_TAG
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# 将 Path 和 str 都保留：RESULTS_PATH 为 Path，RESULTS_PATH_STR 为字符串
PROJ = str(PROJ_DIR)
RESULTS_PATH = RESULTS_DIR
RESULTS_PATH_STR = str(RESULTS_DIR)

print("项目路径:", PROJ)
print("结果路径:", RESULTS_PATH_STR)
print("运行标签:", RUN_TAG)
print(f"确认将要使用的路径是字符串: {RESULTS_PATH_STR}")
# ...existing code...

项目路径: /home/wangxinrui/mattergen
结果路径: /home/wangxinrui/mattergen/data/generated/lps_2025-08-15_17-44-47
运行标签: lps_2025-08-15_17-44-47
确认将要使用的路径是字符串: /home/wangxinrui/mattergen/data/generated/lps_2025-08-15_17-44-47


In [8]:
# Cell 2: Li–P–S 条件生成（写入到 RESULTS_DIR）
# 输出目录作为位置参数；删除未定义的 OUTPUT_DIR/OUTPUT_NAME；修正 --seed
!mattergen-generate "{RESULTS_DIR}" --pretrained-name=chemical_system_energy_above_hull --batch_size=4 --num_batches=1 --properties_to_condition_on='{"chemical_system":"Li-P-S", "energy_above_hull": 0.03}' --diffusion_guidance_factor=2.0 --temperature=1.0 --num_steps=1000 --save_interval=100 --seed=1234

MODELS_PROJECT_ROOT: /home/wangxinrui/mattergen/mattergen
Traceback (most recent call last):
  File "/home/wangxinrui/miniconda3/envs/mattergen/bin/mattergen-generate", line 10, in <module>
    sys.exit(_main())
  File "/home/wangxinrui/mattergen/mattergen/scripts/generate.py", line 103, in _main
    fire.Fire(main)
  File "/home/wangxinrui/miniconda3/envs/mattergen/lib/python3.10/site-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/home/wangxinrui/miniconda3/envs/mattergen/lib/python3.10/site-packages/fire/core.py", line 468, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/home/wangxinrui/miniconda3/envs/mattergen/lib/python3.10/site-packages/fire/core.py", line 684, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/home/wangxinrui/mattergen/mattergen/scripts/generate.py", line 59, in main
    if not os.path.exists(output_path):
  File "/home/wangxinrui/minicond

In [9]:
# 使用 --debug 或 -v 标志查看更多信息
!mattergen-generate --help

MODELS_PROJECT_ROOT: /home/wangxinrui/mattergen/mattergen
INFO: Showing help with the command 'mattergen-generate -- --help'.

NAME
    mattergen-generate - Evaluate diffusion model against molecular metrics.

SYNOPSIS
    mattergen-generate OUTPUT_PATH <flags>

DESCRIPTION
    Evaluate diffusion model against molecular metrics.

POSITIONAL ARGUMENTS
    OUTPUT_PATH
        Type: str
        Path to output directory.

FLAGS
    --pretrained_name=PRETRAINED_NAME
        Type: Optional[Optional]
        Default: None
    -m, --model_path=MODEL_PATH
        Type: Optional[str | None]
        Default: None
        Path to DiffusionLightningModule checkpoint directory.
    -b, --batch_size=BATCH_SIZE
        Type: int
        Default: 64
    -n, --num_batches=NUM_BATCHES
        Type: int
        Default: 1
    --config_overrides=CONFIG_OVERRIDES
        Type: Optional[list[str] | None]
        Default: None
        Overrides for the model config, e.g., `model.num_layers=3 model.hidden_dim=

In [10]:
# Cell 3: 下载参考集（首次运行或缺文件时）
!git lfs pull -I data-release/alex-mp/reference_MP2020correction.gz --exclude=""

In [11]:
# Cell 4: 评估（松弛 + 指标）
!mattergen-evaluate --structures_path="{RESULTS_DIR}" --relax=True --structure_matcher=disordered --save_as="{RESULTS_DIR}/metrics.json"

MODELS_PROJECT_ROOT: /home/wangxinrui/mattergen/mattergen
2025-08-15 17:45:04.612 | INFO     | mattersim.forcefield.potential:from_checkpoint:877 - Loading the pre-trained mattersim-v1.0.0-1M.pth model
0it [00:00, ?it/s]Traceback (most recent call last):
  File "/home/wangxinrui/miniconda3/envs/mattergen/bin/mattergen-evaluate", line 10, in <module>
    sys.exit(_main())
  File "/home/wangxinrui/mattergen/mattergen/scripts/evaluate.py", line 60, in _main
    fire.Fire(main)
  File "/home/wangxinrui/miniconda3/envs/mattergen/lib/python3.10/site-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/home/wangxinrui/miniconda3/envs/mattergen/lib/python3.10/site-packages/fire/core.py", line 468, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/home/wangxinrui/miniconda3/envs/mattergen/lib/python3.10/site-packages/fire/core.py", line 684, in _CallAndUpdateTrace
    component = fn(*varargs, **kwar

In [12]:
# Cell 5: 从 metrics.json 筛选稳定 Li–P–S 候选
import json, csv, re

metrics_file = RESULTS_DIR / "metrics.json"

with open(metrics_file, "r") as f:
    metrics = json.load(f)

items = metrics if isinstance(metrics, list) else metrics.get("items", [])

def has_lps(formula):
    return all(x in formula for x in ["Li","P","S"])

rows = []
for i, it in enumerate(items):
    cif_name = it.get("cif_filename") or it.get("cif_file") or f"struct_{i}.cif"
    formula = it.get("formula") or it.get("reduced_formula") or "Unknown"
    ehull = it.get("energy_above_hull") or it.get("e_above_hull") or None
    if formula == "Unknown":
        m = re.search(r'([A-Z][a-z]?\d*)+', cif_name)
        if m:
            formula = m.group(0)
    if ehull is None:
        continue
    if has_lps(formula) and float(ehull) <= 0.05:
        rows.append({
            "cif_file": cif_name,
            "source_file": RUN_TAG,
            "formula": formula,
            "energy_above_hull": float(ehull)
        })

rows.sort(key=lambda r: r["energy_above_hull"])

candidates_csv = RESULTS_DIR / "candidates.csv"
with open(candidates_csv, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=["cif_file","source_file","formula","energy_above_hull"])
    w.writeheader()
    w.writerows(rows)

print(f"候选数量: {len(rows)}")
print(f"已写入: {candidates_csv}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/wangxinrui/mattergen/data/generated/lps_2025-08-15_17-44-47/metrics.json'

In [ ]:
# Cell 6: 可视化
!python visualize_structures.py --candidates "{candidates_csv}" --top 24 --output_dir "{PROJ}/results/visualizations/{RUN_TAG}" --format html --bonding crystal --supercell auto --plot_size 1000 --verbose
print("可视化输出目录:", f"{PROJ}/results/visualizations/{RUN_TAG}")